# ADASS 2024 Demonstration
Manon Marchand$^1$

1. Strasbourg Astronomical Data Centre (CDS)

This notebook follows the presentation given during ADASS 2024 conference in Malta, on November 14.
You can run it by installing the packages described in the file `requirements.txt`:

`pip install -r requirements.txt`

---

Let's import `ipyaladin` (the star of the show) and `sidecar` (to pop ipyaladin on the side of the notebook rather than in between cells):


In [ ]:
from ipyaladin import Aladin
from sidecar import Sidecar

## 1. Instantiation on the side of the notebook

This will only work in JupyterLab. For other notebooks platforms, reach their documentation to see how/wether this can be done.

In [ ]:
aladin = Aladin()
with Sidecar(title="aladin_output"):
    display(aladin)

Now that the widget is opened on the side, we can resize the window as needed.
The view can be zoomed in and out, or panned. This is a fully functionnal [Aladin Lite](https://github.com/cds-astro/aladin-lite) instance!

## 2. Communication between python and ipyaladin

The added benefit here is that it is connected to the python kernel. We can send information from `ipyaladin` to Python and *vice-versa*.

In [ ]:
aladin.target = "M1"

We sent the string "M1" and the center of the view centered on the Crab Nebulae. If you cannot see it, try zooming in and out.

But `ipyaladin` can also receive some `astropy`-specific objects... and this is the whole subject of this demonstration!

## 3. Astropy SkyCoords and Angle

Let's start by sending a `astropy.coordinates.SkyCoord` object instead of a string:

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
coo_perseus = SkyCoord.from_name("Perseus Cluster")

In [ ]:
aladin.target = coo_perseus

And voilà, we are centered on the Perseus Cluster. Let's also change the field of view with an `astropy.coordinates.Angle` object:

In [ ]:
import astropy.units as u
from astropy.coordinates import Angle

In [ ]:
aladin.fov = Angle(25 * u.arcmin)

The other way around works too. We can zoom in and out until we think we have a rough estimate of the size of the galaxy cluster, and retrieve this value:

In [ ]:
estimated_dimension = aladin.fov
estimated_dimension

## 4. Drawing astropy-regions

Let's plot the estimation of the cluster dimension thanks to an `astropy-region`:

In [ ]:
from regions import CircleSkyRegion

In [ ]:
aladin.add_graphic_overlay_from_region(
    CircleSkyRegion(coo_perseus, estimated_dimension),
    color="salmon", line_dash=[5], 
    name="cluster_dimension"
)

## 5. Adding FITS files to the view

FITS files can either be dragged-and-dropped into the ipyaladin view (like with any Aladin Lite instance), or added programmatically with `add_fits`:

In [ ]:
aladin.add_fits("chandra.fits", name="chandra")

## 6. Working with MOCs

### 6.1 Drawing MOCs on the view

MOCs can be interactively drawn on ipyaladin with the menu `stack` > `+ Overlays` (click on the `+`) > `MOC` >  `From selection`... and then chose your type of selection.
Here, for example, we can draw an appriximated contour for the shockwave visible in the `chandra.fits` image.

### 6.2 Sending MOCs from python to the widget

The widget can also display MOCs:

In [ ]:
from mocpy import MOC

Let's get the MOC describing the coverage on a VizieR catalog.

In [ ]:
gaia_galcan_coverage = MOC.from_vizier_table(
    "I/356/galcand"
)

The result is a `mocpy.MOC` object. We can add it the the widget.

In [ ]:
aladin.add_moc(gaia_galcan_coverage, 
               color="seagreen",
               name="gaia_galcan")

This method also accepts MOCs as ASCII, JSON, URL, or FITS files.

## 7. Changing the survey

### 7.1 With the graphic interface

More surveys can be found with the graphic interface of Aladin Lite. To do so, go in `stack` > `+ Surveys` (click on the `+`) > `Browse HiPS`.
Some filters can be added to restric the results by cecking the box next to `Filter`.

We can for example find that ASKAP offers data in the area by selecting the filters `Inside View` and `Regime` = `Radio`.

### 7.2 With astroquery.MOCServer

In [ ]:
from astroquery.mocserver import MOCServer

Let's look for datasets with the word `Euclid` in their description:

In [ ]:
MOCServer.find_datasets(meta_data="*Euclid*")

In [ ]:
aladin.survey = "CDS/P/Euclid/ERO/VIS"

The view changed from the default survey to the Euclid one.

## 8. Getting a FITS of the current view

This uses the [Hips2FITS](https://alasky.cds.unistra.fr/hips-image-services/hips2fits) image cutout service.

In [ ]:
euclid_cutout = aladin.get_view_as_fits()[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.imshow(np.flipud(euclid_cutout.data.astype("short")),
           cmap="Greys_r", norm="asinh", vmin=10, vmax=50)

The WCS of the current view can also be read at all times:

In [ ]:
aladin.wcs

## 9. Working with catalogs

### 9.1 Adding catalogs to the widget

In [ ]:
from astroquery.vizier import Vizier

In [ ]:
extra_galactic = Vizier(catalog="I/356/galcand", 
                        row_limit=-1).query_region(
    coo_perseus,
    radius=estimated_dimension
)[0]
extra_galactic

In [ ]:
aladin.add_table(
    extra_galactic[extra_galactic["Class"] 
    == "GALAXY"], 
    shape="circle",
    source_size=15, 
    color="lightblue",             
    name="galaxies")

In [ ]:
aladin.add_table(
    extra_galactic[extra_galactic["Class"] 
    != "GALAXY"], 
    shape="circle",
    source_size=15, 
    color="pink",             
    name="galaxies")

### 9.2 Making a sub-selection

To get a sub-selection of these two tables, right click on the widget, and chose `Select Sources` in the context menu that appears.
The last selection is stored as a list of `astropy.Table` in the attribute `aladin.selected_objects`.

If you've selected sources from the two tables, you can recover them with:

In [ ]:
aladin.selected_objects[0]

In [ ]:
aladin.selected_objects[1]

## Conclusion

This was a brief presentation of the features of `ipyaladin` that makes it compatible with the astropy ecosystem. Namely, we've seen that the following objects/libraries are currently supported:

- astropy.coordinates.SkyCoord
- astropy.coordinates.Angle
- astropy-regions (astropy affiliated)
- astropy.io.fits
- mocpy.MOC (astropy affiliated)
- astropy.Table

If you need other objects, don't hesitate to open an issue in `ipyaladin`'s repository https://github.com/cds-astro/ipyaladin

### Useful links

- [ipyaladin's documentation](https://cds-astro.github.io/ipyaladin/)
- [CDS's reporitory of tutorials](https://github.com/cds-astro/tutorials) (they showcase ipyaladin too)